In [1]:
from data.function import Stock,Categories
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import pandas as pd
from datetime import datetime

In [2]:
def convert_interval(data,period):
    if period not in ["MS","QS"]:
         raise Exception("Interval offset is not support. The support interval offset are 'MS' and 'QS'")
    # Use the resample function to get the open price on the first day of the month
    new_open = data['open'].resample(period).first()

    # Use the resample function to get the close price on the last day of the month
    new_close = data['close'].resample(period).last()

    # Use the resample function to get the maximum and minimum values for each month
    new_high = data['high'].resample(period).max()
    new_low = data['low'].resample(period).min()

    new_ma50 = data['MA50'].resample(period).mean()
    new_ma200 = data['MA200'].resample(period).mean()
    # Use the resample function to get the total volume for each month
    new_volume = data['volume'].resample(period).sum()
    # Create a new DataFrame with the monthly stock prices
    new_prices = pd.DataFrame({'open': new_open,
                                    'high': new_high,
                                    'low': new_low,
                                    'close': new_close,                                
                                    'volume': new_volume,
                                    'MA50': new_ma50,
                                    'MA200': new_ma200,
                                    })
    new_prices = new_prices.reset_index()
    return new_prices

def get_close_date(data,frequency):
    if frequency not in ["1d","1h"]:
         raise Exception("This frequency is not support. The support frequency are '1d' and '1h'")
    # grab first and last observations from df.date and make a continuous date range from that
    dt_all = pd.date_range(start=data['x'].iloc[0],end=data['x'].iloc[-1], freq = frequency)

    # check which dates from your source that also accur in the continuous date range
    dt_obs = [d.strftime("%Y-%m-%d %H:%M:%S") for d in data['x']]

    # isolate missing timestamps
    dt_breaks = [d for d in dt_all.strftime("%Y-%m-%d %H:%M:%S").tolist() if not d in dt_obs]
    dt_breaks = pd.to_datetime(dt_breaks)
    return dt_breaks


def candle_plot_1_pic(data,close_day,name):
    #Candlestick Chart
    candle = go.Candlestick(**data.drop(columns=['volume','MA50','MA200']),yaxis = 'y2',name=name)

    #Moving Average 50
    MA50 = go.Scatter(x=data.x, y=data.MA50, line=dict(color='cyan', width=1.5),yaxis = 'y2',name='MA50')

    #Moving Average 200
    MA200 = go.Scatter(x=data.x, y=data.MA200, line=dict(color='#E377C2', width=1.5),yaxis = 'y2',name='MA200')

    #Volume
    volume = go.Bar(x=data.x,y=data.volume,marker={ "color": "gray"},yaxis = 'y',name='Volume')

    trace = [candle,MA50,MA200,volume]

    layout = {
    "xaxis": {"rangeselector": {
        "x": 0, 
        "y": 0.9, 
        "font": {"size": 13}, 
        "visible": True, 
        "bgcolor": "rgba(150, 200, 250, 0.4)", 
        }}, 
    "yaxis": {
        "domain": [0, 0.2], 
        "showticklabels": False
    }, 
    "legend": {
        "x": 0.3, 
        "y": 0.9, 
        "yanchor": "bottom", 
        "orientation": "h"
    }, 
    "margin": {
        "b": 40, 
        "l": 40, 
        "r": 40, 
        "t": 40
    }, 
    "yaxis2": {"domain": [0.2, 0.8]}, 
    "plot_bgcolor": "rgb(250, 250, 250)",
    "title":name+" Candlestick Chart"
    }

    fig = go.Figure(data=trace, layout=layout)
    if len(data['x']) <= 1:# If data have 1 data or less than
        pass
    elif str(data['x'][1]- data['x'][0]) == '0 days 01:00:00':# If interval is 1 hour
        fig.update_xaxes(rangebreaks=[dict(dvalue = 60*60*1000, values=close_day)])
    elif data['x'][1] - data['x'][0] <= pd.Timedelta(days=7):# If interval is less than 7 day
        fig.update_xaxes(rangebreaks=[dict(values=close_day)])
    fig.update_layout(hovermode='x')
    fig.show()

In [3]:
start_date = '2010-12-01'
end_date = '2023-01-14'

# Get stock price
plot_interval = 'monthly' #['hourly','daily','monthly','quarterly']

pull_interval,interval_symbol = '',''

if plot_interval == 'hourly':
    pull_interval = '1h'
    interval_symbol = 'H'
elif plot_interval in ['daily','monthly','quarterly']:
    pull_interval = '1d'
    if plot_interval == 'monthly':
        interval_symbol = 'MS'
    elif plot_interval == 'quarterly':
        interval_symbol = 'QS'
        
stock_symbol = "PTT"
PTT = Stock(stock_symbol)
stock_price = PTT.get_all_stock_price(interval=pull_interval,start=start_date)

stock_price_df = pd.DataFrame(stock_price)
stock_price_df = stock_price_df.rename(columns={0: 'x', 1: 'open', 2: 'high', 3: 'low', 4: 'close',5:'volume'})
stock_price_df['x'] = pd.to_datetime(stock_price_df['x'])
stock_price_df['MA50'] = stock_price_df.close.rolling(50).mean()
stock_price_df['MA200'] = stock_price_df.close.rolling(200).mean() 
stock_price_df.set_index('x', inplace=True)# Set the Date column as the index of the DataFrame

if plot_interval in ['hourly','daily']:
    stock_price_df.reset_index(inplace=True)
else:
    stock_price_df = convert_interval(stock_price_df,interval_symbol)# Convert daily data to monyhly or quarterly


close_day = get_close_date(stock_price_df,pull_interval)# Find Close date

candle_plot_1_pic(stock_price_df,close_day,stock_symbol)

In [36]:
PTT = Stock("PTT")
fs = PTT.financial_statement()

fig = go.Figure()
fig.add_trace(go.Scatter(x=fs["year"],y=fs["กำไรสุทธิ"], line=dict(width=2),
                    name='markers'))

fig.update_layout(
    title="Net Profit",
    font=dict(size=16),
)
fig.show()

In [37]:
PTT = Stock("PTT")
fs = PTT.financial_statement()

fig = go.Figure()
fig.add_trace(go.Bar(x=fs["year"],y=fs["ROA%"],name='ROA%'))
fig.add_trace(go.Bar(x=fs["year"],y=fs["ROE%"],name='ROE%'))
# Add a title to the plot
fig.update_layout(
    title="ROA% ROE% by Year"
)

fig.show()

In [38]:
PTT = Stock("KBANK")
fs = PTT.financial_statement()
print (fs)
fig = go.Figure(go.Treemap(
    labels=fs['year'],
    parents=[""]*len(fs),
    values=fs['ROA%'],
    branchvalues="total",
    hovertext=["year: {}<br>ROA%: {:,.2f}".format(row[0], row[9]) for row in fs.to_numpy()]
))

# Add a title to the plot
fig.update_layout(
    title="ROA% by Year"
)

# Show the plot
fig.show()

   year  สินทรัพย์รวม  หนี้สินรวม  ส่วนของผู้ถือหุ้น  \
0  2561    3155090.81  2737268.89          376297.55   
1  2562    3293888.99  2840174.27          406357.62   
2  2563    3658797.76  3167511.71          439669.63   
3  2564    4103398.52  3566723.50          476713.71   
4  2565    4229795.37  3672186.88          500247.56   

   มูลค่าหุ้นที่เรียกชำระแล้ว  รายได้รวม  กำไรขาดทุน  กำไรสุทธิ  กำไรต่อหุ้น  \
0                    23932.60  182231.62    11704.96   38459.12        16.07   
1                    23932.60  185002.38    16818.44   38726.74        16.18   
2                    23693.28  176921.41     9234.39   29487.12        12.42   
3                    23693.28  258001.38     5662.07   38052.72        15.77   
4                    23693.28  198404.22      270.26   32578.69        13.43   

   ROA%   ROE%  กำไรสุทธิ%  
0  2.64  10.61       24.26  
1  2.52   9.90       23.48  
2  1.72   6.97       19.04  
3  1.75   8.30       16.37  
4  1.71   8.81       16.79  


In [3]:
import plotly.express as px
import pandas as pd
from scipy import stats

# generate some random data
data = {'x': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10],
        'y': [2, 4, 1, 3, 5, 7, 8, 9, 10, 11]}
df = pd.DataFrame(data)

# # fit a linear regression model
# slope, intercept, r_value, p_value, std_err = stats.linregress(df['x'], df['y'])

# create the scatter plot
fig = px.scatter(df, x='x', y='y')

# add the regression line
fig.add_trace(px.scatter(df, x='x', y='y', trendline='ols'))


# show the plot
fig.show()

# # print the slope
# print("The slope of the regression line is: ", slope)


ValueError: 
    Invalid element(s) received for the 'data' property of 
        Invalid elements include: [Figure({
    'data': [{'hovertemplate': 'x=%{x}<br>y=%{y}<extra></extra>',
              'legendgroup': '',
              'marker': {'color': '#636efa', 'symbol': 'circle'},
              'mode': 'markers',
              'name': '',
              'orientation': 'v',
              'showlegend': False,
              'type': 'scatter',
              'x': array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10], dtype=int64),
              'xaxis': 'x',
              'y': array([ 2,  4,  1,  3,  5,  7,  8,  9, 10, 11], dtype=int64),
              'yaxis': 'y'},
             {'hovertemplate': ('<b>OLS trendline</b><br>y = 1.' ... ' <b>(trend)</b><extra></extra>'),
              'legendgroup': '',
              'marker': {'color': '#636efa', 'symbol': 'circle'},
              'mode': 'lines',
              'name': '',
              'showlegend': False,
              'type': 'scatter',
          